---

# Interpretable Machine Learning: Shapley Values
**United Lunch & Learn**: June 6, 2019

_Author: Carleton Smith_

---

## Tutorial Outline

- Install/import packages
- Acquire data
- Initial EDA
- Prepare dataset
- Overview of Shapley Values
    - One
    - Two
    - Three

---
## Install Packages

In [ ]:
import sys
!conda install -yc conda-forge --prefix {sys.prefix} shap

---
## Import Packages

In [1]:
import os
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer

plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['font.size'] = 12
plt.style.use("fivethirtyeight")

---
## Acquire Data

In [2]:
adult = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
                    na_values=' ?',
                    header=None)

If the above line hangs, then uncomment the line below

In [3]:
# adult = pd.read_csv('./datasets/adult.data.txt', header=None, na_values=' ?')

---
## Quick Preprocessing/EDA

- Add column headers
- Understand dataset
    - how many rows/columns?
    - what does a row represent?
    - what is our target variable?
- Check for missing values
- Check data types
- Check for unbalanced target variable

In [4]:
adult.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Add Column Headers

**FEATURES**

1. `age`: continuous.
2. `workclass`: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
3. `fnlwgt`: continuous.
4. `education`: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
5. `education-num`: continuous.
6. `marital-status`: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
7. `occupation`: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
8. `race`: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
9. `sex`: Female, Male.
10. `capital-gain`: continuous.
11. `capital-loss`: continuousm
12. `hours-per-week`: continuous.
13. `native-country`: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [5]:
features = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education_num',
    'marital_status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital_gain',
    'capital_loss',
    'hours_per_week',
    'native_country',
    'income',
]

In [6]:
# assign column names
adult.columns = features
adult.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [7]:
# how many rows a columns?
adult.shape

(32561, 15)

In [8]:
# any missing values?
adult.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education_num        0
marital_status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     583
income               0
dtype: int64

In [9]:
# what are the data types?
adult.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
income            object
dtype: object

In [10]:
# what is the distribution of our target variable?
adult['income'].value_counts()

 <=50K    24720
 >50K      7841
Name: income, dtype: int64

---
## Preprocessing

In the interest of time, I packaged these preprocessing steps into `Pipelines`.

**PREPROCESSING STEPS**
1. Separate target variable from features - sklearn requires this.
2. Peform a train-test split - Always do this before manipulating dataset
3. With training data:
    - **SEPARATE** numeric columns from categorical ones
    - **NUMERIC DF** preprocessing:
        - Replace nan values
        - Standardize features
   
    - **CATEGORICAL DF** preprocessing:
        - Replace nan values
        - Create dummy variables
    - **CONCATENATE** numeric and categorical DF
    - **ENCODE** target variable
<br>
<br>
4. Package these steps into a `Pipeline`

In [11]:
# make a list of numeric and categorical column names
num_cols = [col for col in adult.columns if adult[col].dtype != 'object']
cat_cols = [col for col in adult.columns if col not in num_cols + ['income']]

# separate features from target variable
X = adult.drop('income', axis=1)
y = adult['income']

# perform a train-test split.... why? stratify on y?
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    stratify=y,
    random_state=24
)

def feature_extractor(df):
    return df.drop('income', axis=1)


def categorical_extractor(df):
    return df.select_dtypes(include=['object'])


def numeric_extractor(df):
    return df.select_dtypes(exclude=['object'])

# create custom transformers
cat_transformer = FunctionTransformer(categorical_extractor, validate=False)
num_transformer = FunctionTransformer(numeric_extractor, validate=False)

# make numeric pipe
num_pipe = Pipeline([
    ('numeric_transformer', num_transformer),
    ('num_im', SimpleImputer(strategy='median')),
    ('StandardScaler', StandardScaler())
])

# make categorical pipe
cat_pipe = Pipeline([
    ('cat_transformer', cat_transformer),
    ('cat_im', SimpleImputer(strategy='most_frequent')),
    ('OrdinalEncoder', OrdinalEncoder())
])


# make FeatureUnion
feat_union = FeatureUnion([
    ('num_pipe', num_pipe),
    ('cat_pipe', cat_pipe)
])

# make final feature pipe
feature_pipe = Pipeline([
    ('feat_union', feat_union)
])

#### Use this pipeline to _fit_ and _transform_ `X_train`

In [12]:
# fit and transform training data
X_train_prepared = pd.DataFrame(
    feature_pipe.fit(X_train).transform(X_train),
    index=X_train.index,
    columns=X_train.columns)
X_train_prepared.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
10348,-0.924812,0.012344,1.133671,-0.146749,-0.214716,-0.843279,3.0,9.0,4.0,7.0,1.0,4.0,1.0,38.0
11062,1.712486,2.162427,-1.198581,-0.146749,-0.214716,-1.572986,3.0,1.0,5.0,7.0,1.0,2.0,0.0,38.0
25734,-1.144586,2.329859,-0.421164,-0.146749,-0.214716,0.616136,3.0,11.0,4.0,11.0,1.0,4.0,1.0,38.0
401,-0.778295,-0.763740,-0.032455,-0.146749,-0.214716,-0.032493,3.0,15.0,4.0,7.0,1.0,4.0,0.0,38.0
28063,1.126420,1.600147,1.133671,0.844477,-0.214716,1.183686,4.0,9.0,2.0,3.0,0.0,4.0,1.0,38.0


In [13]:
# transform testing data
X_test_prepared = pd.DataFrame(
    feature_pipe.transform(X_test),
    index=X_test.index,
    columns=X_test.columns)
X_test_prepared.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
2093,0.906645,-0.129961,1.133671,-0.146749,-0.214716,0.535058,3.0,9.0,2.0,11.0,0.0,4.0,1.0,38.0
29473,-1.437619,1.329928,-1.198581,-0.146749,-0.214716,-0.032493,3.0,1.0,5.0,7.0,3.0,4.0,0.0,38.0
14123,0.174062,1.430095,-2.753415,-0.146749,-0.214716,-0.032493,3.0,4.0,2.0,9.0,0.0,4.0,1.0,25.0
10193,0.906645,-1.536521,-0.421164,-0.146749,-0.214716,-0.032493,0.0,11.0,2.0,12.0,0.0,4.0,1.0,38.0
18789,0.320579,1.045784,-0.032455,-0.146749,-0.214716,-0.032493,3.0,15.0,2.0,0.0,0.0,4.0,1.0,38.0


#### Use `LabelEncoder` to transform the `income` to be numeric

In [14]:
y_train[:5]

10348     <=50K
11062     <=50K
25734     <=50K
401       <=50K
28063      >50K
Name: income, dtype: object

In [15]:
# fit and transform y_train
le = LabelEncoder()
y_train_encoded = pd.Series(le.fit_transform(y_train), index=y_train.index)
y_train_encoded[:5]

10348    0
11062    0
25734    0
401      0
28063    1
dtype: int64

In [16]:
# transform y_test
y_test_encoded = pd.Series(le.transform(y_test), index=y_test.index)
y_test_encoded[:5]

2093     1
29473    0
14123    0
10193    0
18789    0
dtype: int64

#### Calculate Baseline

In [17]:
y_test_encoded.value_counts()[0] / y_test_encoded.value_counts().sum()

0.7592384072064694

## Create an Explainable Model: `GradientBoostingClassifier`

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [19]:
rf = RandomForestClassifier(
    n_estimators=10,
    class_weight='balanced',
    oob_score=True
)
rf.fit(X_train_prepared, y_train_encoded)

/Users/carletonsmith/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/carletonsmith/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:463: RuntimeWarning: divide by zero encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/carletonsmith/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:463: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=None, oob_score=True,
            random_state=None, verbose=0, warm_start=False)

In [20]:
# make predictions for training and test:
y_pred_train = rf.predict(X_train_prepared)
y_pred_test = rf.predict(X_test_prepared)

In [21]:
print('CLASSIFICATION METRICS FOR TRAINING: \n')
print(classification_report(y_train_encoded, y_pred_train))
print('#########################################################\n')

print('CLASSIFICATION METRICS FOR TESTING: \n')
print(classification_report(y_test_encoded, y_pred_test))

CLASSIFICATION METRICS FOR TRAINING: 

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     17303
           1       0.99      0.95      0.97      5489

   micro avg       0.99      0.99      0.99     22792
   macro avg       0.99      0.97      0.98     22792
weighted avg       0.99      0.99      0.99     22792

#########################################################

CLASSIFICATION METRICS FOR TESTING: 

              precision    recall  f1-score   support

           0       0.88      0.94      0.91      7417
           1       0.76      0.58      0.66      2352

   micro avg       0.85      0.85      0.85      9769
   macro avg       0.82      0.76      0.78      9769
weighted avg       0.85      0.85      0.85      9769



In [22]:
accuracy_score(y_test_encoded, y_pred_test)

0.8548469648889344

In [23]:
from sklearn.model_selection import cross_val_score

In [24]:
cross_val_score(rf, X_train_prepared, y_train_encoded, scoring='recall', cv=5)

/Users/carletonsmith/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/carletonsmith/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:463: RuntimeWarning: divide by zero encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/carletonsmith/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:463: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/carletonsmith/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/carletonsmith/anaconda3/lib/python3.6/site-packages/sklearn/ensemble

array([0.58469945, 0.55737705, 0.56739526, 0.58196721, 0.58249772])

#### Print Top 10 Features

In [25]:
feat_imp_lst = list(zip(X_train_prepared.columns, rf.feature_importances_))
feat_lst = sorted(feat_imp_lst, key=lambda x: x[1], reverse=True)
for tup in feat_lst[:10]:
    print(tup)

('capital_gain', 0.1523186888793026)
('workclass', 0.1411485445161116)
('age', 0.13924353923051092)
('race', 0.10222939459653206)
('fnlwgt', 0.1020508069619955)
('education', 0.08451394498593742)
('marital_status', 0.08180095088517161)
('sex', 0.060334385526217435)
('relationship', 0.0328853170054671)
('occupation', 0.03276244222617073)


## Shapley values

The `shap` package includes a C++ optimized implementation for several popular Python tree models.

In [26]:
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_train_prepared)

Scratch Work

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_lst = {
    "n_estimators": np.arange(10, 105, 10),
    "max_depth": [None, 30, 10, 5],
    "class_weight": ['balanced'],
    "oob_score": [True],
}

In [ ]:
rs = RandomizedSearchCV(rf, param_distributions=param_lst, verbose=2)

In [ ]:
rs.fit(X_train_prepared, y_train_encoded)

In [ ]:
rs.best_estimator_